### Introduction
In this exercise you will build three (optionally four) different networks to experiment with one of the applications of recurrent neural networks called Machine Translation. The preprocessing pipeline that prepares the input data for feeding into the neural network models is provided to you. Therefore, in this excercise you focus on building and experimenting with different network architectures. 

Optional: To learn more about recurrent neural networks (such as RNN, GRU, and LSTM) their architectures, the differences between them and their applications you can watch this free tutorial: [Sequences and Recurrent networks Tutorial](https://www.youtube.com/watch?v=87kLfzmYBy8&ab_channel=DeepMind)

In [ ]:
# You may want to upgrade the tensorflow package to the latest verion (currently version 2.3) if you have the proper infrastructure
# !pip install --upgrade tensorflow

In [ ]:
pip install --upgrade tensorflow

### Setup


In [ ]:
# To get access to a GPU instance you can use the `change runtime type` and set the option to `GPU` from the `Runtime` tab  in the notebook
# Checking the GPU availability for the notebook
import tensorflow
tensorflow.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# import collection
import io
import pandas as pd
import numpy as np
from typing import List 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

### Loading and Preprocessing the Data

#### Reading CSV data files as a list of strings, where each string represents a single line from the text


In [ ]:
import csv
from io import TextIOWrapper
from zipfile import ZipFile

In [ ]:
import requests


url = 'https://github.com/murpi/wilddata/blob/master/quests/small_vocab_en.csv.zip?raw=true'
r = requests.get(url, allow_redirects=True)

open('small_vocab_en.csv.zip', 'wb').write(r.content)

url = 'https://github.com/murpi/wilddata/blob/master/quests/small_vocab_fr.csv.zip?raw=true'
r = requests.get(url, allow_redirects=True)

open('small_vocab_fr.csv.zip', 'wb').write(r.content)

1367852

In [ ]:
with ZipFile('small_vocab_en.csv.zip') as zf:
    with zf.open('small_vocab_en.csv', 'r') as infile:
        reader = csv.reader(TextIOWrapper(infile, 'utf-8'), delimiter='\n' )
        english_text = [item for sublist in reader for item in sublist]
        print(f"English text samples: {english_text[0:2]}")

English text samples: ['new jersey is sometimes quiet during autumn , and it is snowy in april .', 'the united states is usually chilly during july , and it is usually freezing in november .']


In [ ]:
with ZipFile('small_vocab_fr.csv.zip') as zf:
    with zf.open('small_vocab_fr.csv', 'r') as infile:
        reader = csv.reader(TextIOWrapper(infile, 'utf-8'), delimiter='\n' )
        french_text = [item for sublist in reader for item in sublist]
        print(f"French translation text samples: {french_text[0:2]}")


French translation text samples: ["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .", 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .']


In [ ]:
# TODO: Data exploration -> calculate and print some statistics on data


#### Tokenizing the data

Since the input data to neural networks needs to be in numerical format, we first turn each sentence into a sequence of word ids using `Tokenizer` function from Keras. Word ids are numerical presentations for words. 

In [ ]:
def tokenize(x: List[str]):
  """Tokenizes sentences into word ids.
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)

  """
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(x)
  t=tokenizer.texts_to_sequences(x)
  return t, tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent)) 

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}
Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


#### Padding Sequences

To be able to batch the sequences of word ids together, all sequences need to be of the same length. Therefore, as a second preprocessing step we make sure that all the English sentences have the same length as their respective French translations using `pad_sequence` function form Keras.

In [ ]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    padding=pad_sequences(x,padding='post',maxlen=length)
    return padding

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


#### Preprocessing Pipeline

As mentioned in the introduction section in this excercise your focus is on building different recurrent neural network architectures. Threfore, in the following cell we provide you with a preprocessing pipeline. You can apply this function to the input data to prepare the data for using in neural networks.

In [ ]:
def preprocess(x, y):
    """
     Preprocess input (x) and target (y)
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_text, french_text)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Preprocessed Data Info')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Preprocessed Data Info
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


### Models

In this section we ask you to build and train Three (optionally four) different neural networks and use them to predict the French Translation of the English sentences.

Make use of the `logits_to_text()` function to transform the logits from output of your networks to a French translation. This helps you to better understand the output of the neural network.

In [ ]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


#### Model 1: Implement a simple RNN network and train the network. 


In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Input,InputLayer, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.callbacks import EarlyStopping

In [ ]:
# Model 1: simple RNN model
EarlyStoping = EarlyStopping(monitor="val_loss",verbose=1,mode='min',patience=3)


def rnn_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers and compile the model. (Hint: Use a GRU network followed by a TimeDistributed dense layer from tensorflow.keras.layers module)
    # What is the input shape of your model? Which activation function do you choose for the dense layer?
    
    model = Sequential( # Complete here
        


    return model

model = rnn_model(preproc_english_sentences_21.shape,max_french_sequence_length,english_vocab_size+1,french_vocab_size+1)

model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(0.005),
              metrics=['accuracy'])

# TODO: Preprocess and reshape the input to work with a basic RNN. (Hint: Use preproc_english_sentences, preproc_french_sentences and reshape function from keras)

# TODO: Train the network you built in rnn_model. What size do you choose for the epochs, batch_size and validation_split?


In [ ]:
# TODO: Print prediction(s) for one or more of the English sentences. Hint: Make use of the `logits_to_text` function for transforming output logits to text. 



'les états unis est généralement froid en septembre et il est généralement agréable en novembre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

#### Model 2: Implement an RNN model using word embeddings

Embeddings are better representation of words. An embedding is a vector representation of a word in n-dimensional space, where n represents the size of the embedding vector.


In [ ]:
# Model 2: Embedding
EarlyStoping = EarlyStopping(monitor="val_loss",verbose=1,mode='min',patience=3)
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers and compile the model. (Hint: Similar to the first model use a GRU network followed by a TimeDistributed dense layer from tensorflow.keras.layers module. 
    # And this time make use of Embedding as well!)
    # What is the input shape of your model? Which activation function do you choose for the dense layer?
    embedding_size = 128  # a common choice but fully arbitrary chosen here 
    model = Sequential( # Complete here
                       

                       
    return model

model_embeding = embed_model(preproc_english_sentences_21.shape,max_french_sequence_length,english_vocab_size+1,french_vocab_size+1) 

histemb=model_embeding.fit(preproc_english_sentences_21,preproc_french_sentences,validation_split=0.2, epochs=100, verbose=1,callbacks=[EarlyStoping])

# TODO: Preprocess and reshape the input to work with a basic RNN. (Hint: Use preproc_english_sentences, preproc_french_sentences and reshape function from keras)
 
# TODO: Train the network you built in rnn_model. What size do you choose for the epochs, batch_size and validation_split?


In [ ]:
# TODO: Print prediction(s) for one or more of the English sentences. Hint: Make use of the `logits_to_text` function for transforming output logits to text. 
# (Hint: If you predict on the same sentence you can compare the result of different model on that specific sentence with each other.)



#### Model 3: Implement a Bidirectional RNN network.

One of the restrictions of RNN networks is that they only see the data from past. However, Bidirectional RNNs can see the data in both directions past and future. Make use of Bidirectionality and experiment if this feature enhances your network's performance or not.

In [ ]:
# Model 3: Bidirectional RNNs
EarlyStoping = EarlyStopping(monitor="val_loss",verbose=1,mode='min',patience=3)
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO:  Build the layers and compile the model. (Hint: Similar to the first model use a GRU network followed by a TimeDistributed dense layer from tensorflow.keras.layers module. 
    # Don't forget to make your network bidirectional!)

    model = Sequential( # Complete here
                       

    return model
model_bidirectional = bd_model(preproc_english_sentences_21.shape,max_french_sequence_length,english_vocab_size+1,french_vocab_size+1) 

histbd=model_bidirectional.fit(preproc_english_sentences_21,preproc_french_sentences,validation_split=0.2, epochs=100, verbose=1,callbacks=[EarlyStoping])


In [ ]:
# TODO: Print prediction(s) for one or more of the English sentences. Hint: Make use of the `logits_to_text` function for transforming output logits to text.
# (Hint: If you predict on the same sentence you can compare the result of different model on that specific sentence with each other.)



#### Model 4: Implement an Encoder-Decoder model (Optional)

In [ ]:

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement

    return model


# OPTIONAL: Train and Print prediction(s)


#### Finally, you can use all the models you built in this exercise to create a model that incorporates embedding and bidirectionality into one model. (Optional)

In [ ]:

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    return model
# OPTIONAL: Train and Print prediction(s)

### Final Notes and TODOs




*   In this exercise we focus on  learning different recurrent network architectures for machine translation, However we don't evaluate the models on a separate test set. To follow best machine learning practices, you can make use of the `sklearn.model_selecttion.train_test_split()` function to create separate training and test datasets. You can then retrain each of the models on the training set and evaluate the prediction accuracy using the hold out set. Observe how the best model performance might change.
* If you don't have any idea about French (like me :D) you can use a translater like [google translat](https://translate.google.com/) to compare the prediction(s) from your models with the respective original English sentence(s).
*   TODO: Finally, you can also try to train and predict your models on other language pairs. As a convenient example, you can use your models to translate from French to English since you already have the data available.



